In [90]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, udf, when, rand

In [2]:
spark = SparkSession.builder.appName("u_to_sb").getOrCreate()

your 131072x1 screen size is bogus. expect trouble
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/01/28 10:38:07 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
shots = spark.read.csv("understat/understat.csv", header=True, inferSchema=True)
shots_sb = spark.read.csv("Data/Datas/shots.csv", header=True, inferSchema=True)

In [9]:
shots = shots.withColumnRenamed("regular_play", "from_rp")

In [14]:
FEATURES = ['from_rp','from_fk','from_corner','from_counter',
            'header','corner_type','fk_type','pk_type',
            'acrobatic_shot','lob_technique',
            'distance_to_goal', 'shot_angle', 'preferred_foot_shot', 'under_pressure',
            'shot_aerial_won','shot_first_time','shot_one_on_one','shot_open_goal','shot_follows_dribble','players_inside_area']

In [27]:
# check if the columns are in the dataframe
for feature in FEATURES:
    if feature not in shots.columns:
        print(f"Column {feature} not in dataframe")

Column from_counter not in dataframe
Column lob_technique not in dataframe
Column under_pressure not in dataframe
Column shot_aerial_won not in dataframe
Column shot_open_goal not in dataframe
Column players_inside_area not in dataframe


In [24]:
shots = shots.withColumnRenamed('situation_FromCorner', 'from_corner')

In [20]:
# corner_type when the shot_location_x = 119.4 and shot_location_y = 79.6 or 0.4
shots = shots.withColumn("corner_type",
                 when((shots.shot_location_x == 119.4) &
                      ((shots.shot_location_y == 79.6) | (shots.shot_location_y == 0.4)), 1).otherwise(0))

In [26]:
shots.show()

+------+---------+------------------+------------------+-----------+------+-----------------+----------------------+------------------+-------+-----------+-------+-------+------------------+----+------------------+----------------+------------------+--------------+-------------------+-------+--------------------+---------------+--------------+---------------+-----------+
|    id|player_id|   shot_location_x|   shot_location_y| lastAction|header|shotType_LeftFoot|shotType_OtherBodyPart|shotType_RightFoot|fk_type|from_corner|from_rp|pk_type|situation_SetPiece|goal|                xG|distance_to_goal|        shot_angle|preferred_foot|preferred_foot_shot|from_fk|shot_follows_dribble|shot_first_time|acrobatic_shot|shot_one_on_one|corner_type|
+------+---------+------------------+------------------+-----------+------+-----------------+----------------------+------------------+-------+-----------+-------+-------+------------------+----+------------------+----------------+------------------+--

In [41]:
shots_sb.filter(col("fk_type") == 1).groupBy('players_inside_area').count().show(20)

+-------------------+-----+
|players_inside_area|count|
+-------------------+-----+
|                 12|    1|
|                  1|  106|
|                 13|    2|
|                  6|  452|
|                  3|  487|
|                  5|  671|
|                  9|   30|
|                  4|  604|
|                  8|   82|
|                  7|  214|
|                 10|    9|
|                 11|    2|
|                  2|  258|
|                  0|   14|
+-------------------+-----+



In [28]:
s = shots

In [29]:
# save the dataframe
shots.write.csv("understats/understat.csv", header=True, mode='overwrite')

In [42]:
# separating corners, free kicks, penalties from the shots dataframe
n_shots = shots.filter((col('corner_type') == 0) & (col('fk_type') == 0) & (col('pk_type') == 0))
sp_shots = shots.filter((col('corner_type') == 1) |(col('pk_type') == 1))
fk_shots = shots.filter(col('fk_type') == 1)

In [47]:
sp_shots = sp_shots.withColumn('players_inside_area',
                    when(sp_shots.corner_type == 1, 0).otherwise(1))

In [98]:
fk_shots.withColumn(
    "lob_technique",
    when(
        (col("fk_type") == 1) & (rand() < 0.0020463847203274215), 1
    ).otherwise(0)).show(5)

+------+---------+-----------------+-----------------+----------+------+-----------------+----------------------+------------------+-------+-----------+-------+-------+------------------+----+------------------+----------------+------------------+--------------+-------------------+-------+--------------------+---------------+--------------+---------------+-----------+-------------+
|    id|player_id|  shot_location_x|  shot_location_y|lastAction|header|shotType_LeftFoot|shotType_OtherBodyPart|shotType_RightFoot|fk_type|from_corner|from_rp|pk_type|situation_SetPiece|goal|                xG|distance_to_goal|        shot_angle|preferred_foot|preferred_foot_shot|from_fk|shot_follows_dribble|shot_first_time|acrobatic_shot|shot_one_on_one|corner_type|lob_technique|
+------+---------+-----------------+-----------------+----------+------+-----------------+----------------------+------------------+-------+-----------+-------+-------+------------------+----+------------------+----------------+--

In [97]:
sp_shots.withColumn(
    "lob_technique",
    when(
        (col("pk_type") == 1) & (rand() < 0.0031982942430703624), 1
    ).otherwise(0)).show(5)

+------+---------+---------------+---------------+----------+------+-----------------+----------------------+------------------+-------+-----------+-------+-------+------------------+----+------------------+----------------+------------------+--------------+-------------------+-------+--------------------+---------------+--------------+---------------+-----------+-------------------+-------------+
|    id|player_id|shot_location_x|shot_location_y|lastAction|header|shotType_LeftFoot|shotType_OtherBodyPart|shotType_RightFoot|fk_type|from_corner|from_rp|pk_type|situation_SetPiece|goal|                xG|distance_to_goal|        shot_angle|preferred_foot|preferred_foot_shot|from_fk|shot_follows_dribble|shot_first_time|acrobatic_shot|shot_one_on_one|corner_type|players_inside_area|lob_technique|
+------+---------+---------------+---------------+----------+------+-----------------+----------------------+------------------+-------+-----------+-------+-------+------------------+----+----------

In [ ]:
n_shots.withColumn(
    "lob_technique",
    when(
        (col("shot_one_on_one") == 1) & (rand() < 0.03534563653425086), 1).otherwise(0)).show(5)

+------+---------+------------------+------------------+----------+------+-----------------+----------------------+------------------+-------+-----------+-------+-------+------------------+----+------------------+----------------+------------------+--------------+-------------------+-------+--------------------+---------------+--------------+---------------+-----------+-------------+
|    id|player_id|   shot_location_x|   shot_location_y|lastAction|header|shotType_LeftFoot|shotType_OtherBodyPart|shotType_RightFoot|fk_type|from_corner|from_rp|pk_type|situation_SetPiece|goal|                xG|distance_to_goal|        shot_angle|preferred_foot|preferred_foot_shot|from_fk|shot_follows_dribble|shot_first_time|acrobatic_shot|shot_one_on_one|corner_type|lob_technique|
+------+---------+------------------+------------------+----------+------+-----------------+----------------------+------------------+-------+-----------+-------+-------+------------------+----+------------------+-------------

In [101]:
n_shots_sb = shots_sb.filter((col('corner_type') == 0) & (col('fk_type') == 0) & (col('pk_type') == 0))

In [142]:
X_values = [x for x in range(0,121,24)]
Y_values = [y for y in range(0,81,16)]
X_values, Y_values

([0, 24, 48, 72, 96, 120], [0, 16, 32, 48, 64, 80])

In [156]:
# combine each two consecutive values of x with each two consecutive values of y
# [0,24] with [0,16], [16,32], [32,48], [48,64], [64,80]
# [24,48] with [0,16], [16,32], [32,48], [48,64], [64,80]

coo = []
for X in range(len(X_values[:-1])):
    for Y in range(len(Y_values[:-1])):
        coo.append((X_values[X], X_values[X+1], Y_values[Y], Y_values[Y+1]))
coo

[(0, 24, 0, 16),
 (0, 24, 16, 32),
 (0, 24, 32, 48),
 (0, 24, 48, 64),
 (0, 24, 64, 80),
 (24, 48, 0, 16),
 (24, 48, 16, 32),
 (24, 48, 32, 48),
 (24, 48, 48, 64),
 (24, 48, 64, 80),
 (48, 72, 0, 16),
 (48, 72, 16, 32),
 (48, 72, 32, 48),
 (48, 72, 48, 64),
 (48, 72, 64, 80),
 (72, 96, 0, 16),
 (72, 96, 16, 32),
 (72, 96, 32, 48),
 (72, 96, 48, 64),
 (72, 96, 64, 80),
 (96, 120, 0, 16),
 (96, 120, 16, 32),
 (96, 120, 32, 48),
 (96, 120, 48, 64),
 (96, 120, 64, 80)]